In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from sqlalchemy import create_engine
from ta import add_all_ta_features
import plotly.graph_objs as go
import pandas_ta as ta

In [2]:
df = yf.download("SPY", period = "60d", interval = "5m")

[*********************100%%**********************]  1 of 1 completed


In [3]:
df["Date"] = pd.to_datetime(df.index ,format='%d.%m.%Y %H:%M:%S')

df=df[df.High!=df.Low]
df.set_index("Date", inplace=True)

In [4]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=30)
df['RSI']=ta.rsi(df.Close, length=10)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
df=df.join(my_bbands)
df

,Open,High,Low,Close,Adj Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
Date,,,,,,,,,,,,,,,
2023-11-13 09:30:00-05:00,439.230011,439.589996,438.790009,438.809906,438.809906,2446754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-13 09:35:00-05:00,438.799988,438.897705,438.600006,438.720001,438.720001,883874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-13 09:40:00-05:00,438.709991,438.885010,438.589996,438.635010,438.635010,821057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-13 09:45:00-05:00,438.640015,438.875000,438.630005,438.839996,438.839996,745655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-13 09:50:00-05:00,438.850006,439.029999,438.613312,438.640015,438.640015,849565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-08 15:35:00-05:00,498.399994,498.420013,498.309998,498.369995,498.369995,809621,498.031603,498.122673,62.315580,0.188481,497.823933,498.173397,498.522860,0.140298,0.781286
2024-02-08 15:40:00-05:00,498.359985,498.459991,498.279999,498.304993,498.304993,514189,498.042324,498.134435,56.934596,0.187268,497.881108,498.204991,498.528873,0.130020,0.654380
2024-02-08 15:45:00-05:00,498.320007,498.350006,498.010010,498.059998,498.059998,971060,498.043017,498.129633,41.813919,0.209086,497.934647,498.222331,498.510014,0.115484,0.217861


In [5]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

  0%|          | 0/4641 [00:00<?, ?it/s]

100%|██████████| 4641/4641 [00:07<00:00, 612.38it/s]
C:\Users\Vishnu\AppData\Local\Temp\ipykernel_15212\3663350097.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0


In [6]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

  0%|          | 0/4641 [00:00<?, ?it/s]

100%|██████████| 4641/4641 [00:13<00:00, 339.31it/s]
C:\Users\Vishnu\AppData\Local\Temp\ipykernel_15212\3699091765.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)


In [7]:
df[df.TotalSignal != 0].head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
74,2023-11-13 15:40:00-05:00,440.570007,440.670013,440.440002,440.489990,440.489990,682534,440.448332,440.619928,40.619889,0.244764,440.496057,440.728969,440.961882,0.105694,-0.013023,2,2
76,2023-11-13 15:50:00-05:00,440.579987,440.779999,440.100006,440.304993,440.304993,1066748,440.448048,440.597803,35.674007,0.305133,440.412325,440.671635,440.930945,0.117689,-0.206957,2,2
77,2023-11-13 15:55:00-05:00,440.299988,440.339996,440.140015,440.200012,440.200012,2367538,440.438321,440.572139,32.651509,0.290111,440.338276,440.618636,440.898996,0.127257,-0.246583,2,2
106,2023-11-14 11:50:00-05:00,449.119995,449.140015,448.670013,448.885010,448.885010,890765,446.210320,447.745110,54.702911,0.386025,448.943140,449.235819,449.528499,0.130301,-0.099306,2,2
107,2023-11-14 11:55:00-05:00,448.869995,448.885010,448.399994,448.434998,448.434998,631308,446.297562,447.789619,45.659688,0.400167,448.772577,449.190820,449.609064,0.186221,-0.403568,2,2
108,2023-11-14 12:00:00-05:00,448.439697,448.720001,448.359985,448.559998,448.559998,705646,446.386285,447.839321,48.297637,0.394431,448.668085,449.148214,449.628342,0.213795,-0.112561,2,2
109,2023-11-14 12:05:00-05:00,448.540009,448.589996,447.959991,448.220001,448.220001,1002995,446.458195,447.863881,42.118363,0.428084,448.493668,449.086214,449.678760,0.263890,-0.230925,2,2
110,2023-11-14 12:10:00-05:00,448.230011,448.279999,448.070007,448.250000,448.250000,690816,446.528462,447.888792,42.835373,0.396928,448.369129,449.038881,449.708634,0.298305,-0.088935,2,2
111,2023-11-14 12:15:00-05:00,448.250000,448.269989,448.084991,448.140015,448.140015,907898,446.591660,447.905000,40.777610,0.366652,448.222859,448.965576,449.708293,0.330857,-0.055771,2,2
112,2023-11-14 12:20:00-05:00,448.140015,448.149994,447.869995,447.989990,447.989990,640397,446.646497,447.910483,38.010175,0.354273,448.065452,448.884908,449.704364,0.365108,-0.046044,2,2


In [8]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

C:\Users\Vishnu\AppData\Local\Temp\ipykernel_15212\2345118794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [10]:
def SIGNAL():
    return df.TotalSignal

In [11]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 10
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=25000, margin=1/30)

C:\Users\Vishnu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

C:\Users\Vishnu\AppData\Local\Temp\ipykernel_15212\1382273915.py:36: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [12]:
bt.run()

Start                                     0.0
End                                    4640.0
Duration                               4640.0
Exposure Time [%]                   33.462616
Equity Final [$]                 25121.314356
Equity Peak [$]                  25165.416417
Return [%]                           0.485257
Buy & Hold Return [%]               13.476246
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.353074
Avg. Drawdown [%]                   -0.037745
Max. Drawdown Duration                 2104.0
Avg. Drawdown Duration             107.605263
# Trades                                227.0
Win Rate [%]                        43.612335
Best Trade [%]                       1.408134
Worst Trade [%]                      -0.50787
Avg. Trade [%]                    